In [1]:
!pip install torch
!pip install dgl
!pip install torch_geometric
!pip install tensorflow

    extract-msg (<=0.29.*)
                 ~~~~~~~^
    extract-msg (<=0.29.*)
                 ~~~~~~~^
    extract-msg (<=0.29.*)
                 ~~~~~~~^
    extract-msg (<=0.29.*)
                 ~~~~~~~^


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import os
from natsort import natsorted
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
import string
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from gensim.models import FastText
import matplotlib.pyplot as plt
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torch_geometric
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense,Bidirectional
from tensorflow.keras.regularizers import l2
import torch
import torch.nn as nn
import torch.nn.functional as F
# import torch_geometric
import numpy as np
from numpy.linalg import norm
from dgl.nn import GraphConv
from IPython.display import FileLink

import os
import zipfile
import numpy as np
from nltk.tokenize import word_tokenize
from natsort import natsorted
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


In [3]:
import os
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from natsort import natsorted
import zipfile
import pandas as pd
import nltk
import string

In [4]:
# Download NLTK stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/priyapatil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/priyapatil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# Define punctuation to remove
stopwords_list = stopwords.words('english')
additional_stopwords = ['33', '52']
stopwords_list.extend(additional_stopwords)

remove = string.punctuation + '’‘–—“”'

# Function to remove punctuation
def pun_remove(data):
    for char in remove:
        data = data.replace(char, ' ')
    return data

In [6]:
class MySentences:
    def __init__(self, root_directory):
        self.root_directory = root_directory

    def __iter__(self):
        # Get a sorted list of subdirectories in the root directory
        file_list = [file for file in os.listdir(self.root_directory) if not file.startswith('.DS_Store')]
        file_list = natsorted(file_list)

        for subdirectory in file_list:
            subdirectory_path = os.path.join(self.root_directory, subdirectory, 'txt')
            if os.path.exists(subdirectory_path):
                # Get a sorted list of files in the subdirectory
                file_list_t = [file for file in os.listdir(subdirectory_path) if not file.startswith('.DS_Store')]
                file_list_t = natsorted(file_list_t)

                for filename in file_list_t:
                    file_path = os.path.join(subdirectory_path, filename)
                    with open(file_path, 'r') as file:
                        A = []
                        for line in file:
                            line = line.lower()
                            line = pun_remove(line)  # Custom function to remove punctuation
                            line = word_tokenize(line)  # Tokenize the line
                            # Remove stopwords
                            line_Without_stopwords = [word for word in line if word not in stopwords_list]
                            if line_Without_stopwords:
                                A.append(line_Without_stopwords)
                        # Flatten and combine tokens into a sentence
                        A = list(np.concatenate(A))
                        yield ' '.join(A)


In [7]:
# Vocabulary and Tokenizer
class CustomTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab
        self.pad_token = '[PAD]'
        self.unk_token = '[UNK]'
        self.cls_token = '[CLS]'
        self.mask_token = '[MASK]'

        # Add special tokens to vocab
        self.vocab[self.pad_token] = len(self.vocab)
        self.vocab[self.unk_token] = len(self.vocab)
        self.vocab[self.cls_token] = len(self.vocab)
        self.vocab[self.mask_token] = len(self.vocab)

        self.reverse_vocab = {v: k for k, v in self.vocab.items()}

    def encode(self, text):
        tokens = text.split()
        token_ids = [self.vocab.get(token, self.vocab[self.unk_token]) for token in tokens]
        return torch.tensor(token_ids, dtype=torch.long)

    def decode(self, token_ids):
        tokens = [self.reverse_vocab.get(id.item(), self.unk_token) for id in token_ids]
        return " ".join(tokens)


In [8]:
# Dataset
class TextDataset:
    def __init__(self, texts, tokenizer, max_len=50):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def process_all(self):
        token_ids_list = []
        for text in self.texts:
            token_ids = self.tokenizer.encode(text)
            token_ids = token_ids[:self.max_len]
            token_ids = torch.cat([
                token_ids,
                torch.tensor([self.tokenizer.vocab[self.tokenizer.pad_token]] * (self.max_len - len(token_ids)))
            ])
            token_ids_list.append(token_ids)

        # Stack all token IDs
        return torch.stack(token_ids_list)


In [9]:
class BERTEmbeddingExtractor(nn.Module):
    def __init__(self, vocab_size, embedding_dim, max_len):
        super(BERTEmbeddingExtractor, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.max_len = max_len

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=4),
            num_layers=1,
        )
        self.fc = nn.Linear(embedding_dim, vocab_size)

    def forward(self, x):
        x = self.embeddings(x)
        x = self.transformer(x)
        x = self.fc(x)
        return x

    def extract_embeddings(self, x):
        """
        Extract embeddings from the embedding layer.
        """
        return self.embeddings(x)


In [10]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/priyapatil/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [13]:
# Prepare data
zip_file_path = '/Users/priyapatil/Desktop/Image_data'
# extract_to = '/Extracted_image_data'
sentences = MySentences(zip_file_path)
docs = [''.join(sentence) for sentence in sentences]
print(docs[:3])
print(f"Total number of documents: {len(docs)}")


['hail durga tara destroyer sorrows invincible strong southern winds nagendranandini daughter king mountains chandi ten arms stretch ten directions third eye shines brightly middle forehead lakshmi saraswati left kartik ganesh along lion asura joya bijoya accompany mother', 'one day goddess durga bestowed grace revealed cache precious gems pomegranate grove kalketu found treasure pomegranate tree cleared forest named gujarat established town two pots gems remained kalketu filled jars sealing carefully carried jars home beautiful khullona phullora waited doorway thatched hut twelve years ascetic languishing jail meanwhile khullona given birth son srimonto srimonto grew bright young man wished go search father son light life cried khullona leave even moment shall feel abandoned must go pray durga invoked chandi durga appeared khullona handed srimonto goddess', 'shouting hail durga srimonto boarded boat boatmen set sail sailing peacefully waters magora struck misfortune caught terrible st

In [18]:
# Build vocabulary
all_tokens = [token for doc in docs for token in word_tokenize(doc)]
word_counts = Counter(all_tokens)
vocab = {word: idx for idx, (word, _) in enumerate(word_counts.most_common())}

tokenizer = CustomTokenizer(vocab)
dataset = TextDataset(docs, tokenizer)

In [19]:
# Process all data
inputs = dataset.process_all()
inputs = inputs.long()

In [20]:
# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTEmbeddingExtractor(vocab_size=len(tokenizer.vocab), embedding_dim=100, max_len=50).to(device)


/Users/priyapatil/anaconda3/lib/python3.11/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [21]:
# Extract embeddings for the entire dataset
inputs = inputs
embeddings = model.extract_embeddings(inputs)
embeddings = embeddings.detach().cpu().numpy()


In [22]:
# Mean pooling
sequence_embeddings = embeddings.mean(axis=1)


In [23]:
# Save embeddings to CSV
embeddings_df = pd.DataFrame(sequence_embeddings)
embeddings_df.to_csv("embeddings.csv", index=False)
print("Embeddings saved to embeddings.csv")

Embeddings saved to embeddings.csv


In [24]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [25]:
labels = pd.read_csv("/Users/priyapatil/Desktop/RecSys/ProjectData/AnimalLabels.csv")
labels['majority_vote'] = labels.mode(axis=1, numeric_only=True).astype(int)
Label_df = labels[['majority_vote']]

train_df, test_df, train_labels, test_labels = train_test_split(
    embeddings_df, Label_df, test_size=0.2, random_state=42
)

In [26]:
print(train_df.shape,test_df.shape,train_labels.shape,test_labels.shape)

similarity_matrix = cosine_similarity(train_df)
threshold = 0.5  # Define threshold for creating edges

(84, 100) (22, 100) (84, 1) (22, 1)


In [27]:
# Create edges based on similarity threshold
src, dst = [], []
for i in range(similarity_matrix.shape[0]):
    for j in range(i + 1, similarity_matrix.shape[1]):
        if similarity_matrix[i, j] > threshold:
            src.append(i)
            dst.append(j)

In [28]:
g = dgl.graph((src + dst, dst + src))  # Undirected graph

# Add node features (embeddings) to the graph
g.ndata['feat'] = torch.tensor(train_df.values, dtype=torch.float32)

# Random labels for the training graph (replace with actual labels)
num_nodes = train_df.shape[0]

In [29]:
class GraphSAGENet(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GraphSAGENet, self).__init__()
        self.conv1 = dgl.nn.SAGEConv(in_feats, hidden_feats, aggregator_type='mean')
        self.conv2 = dgl.nn.SAGEConv(hidden_feats, out_feats, aggregator_type='mean')

    def forward(self, g, inputs):
        x = self.conv1(g, inputs)
        x = F.relu(x)
        x = self.conv2(g, x)
        return F.log_softmax(x, dim=1)
    
model = GraphSAGENet(in_feats=train_df.shape[1], hidden_feats=128, out_feats=3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [30]:
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    out = model(g, g.ndata['feat'])
    train_labels_tensor = torch.tensor(train_labels.values, dtype=torch.long)
    train_labels_tensor = train_labels_tensor.argmax(dim=1)
    loss = F.nll_loss(out, train_labels_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 1.8684
Epoch 2, Loss: 0.0104
Epoch 3, Loss: 0.0001
Epoch 4, Loss: 0.0000
Epoch 5, Loss: 0.0000
Epoch 6, Loss: 0.0000
Epoch 7, Loss: 0.0000
Epoch 8, Loss: 0.0000
Epoch 9, Loss: 0.0000
Epoch 10, Loss: 0.0000
Epoch 11, Loss: 0.0000
Epoch 12, Loss: 0.0000
Epoch 13, Loss: 0.0000
Epoch 14, Loss: 0.0000
Epoch 15, Loss: 0.0000
Epoch 16, Loss: 0.0000
Epoch 17, Loss: 0.0000
Epoch 18, Loss: 0.0000
Epoch 19, Loss: 0.0000
Epoch 20, Loss: 0.0000
Epoch 21, Loss: 0.0000
Epoch 22, Loss: 0.0000
Epoch 23, Loss: 0.0000
Epoch 24, Loss: 0.0000
Epoch 25, Loss: 0.0000
Epoch 26, Loss: 0.0000
Epoch 27, Loss: 0.0000
Epoch 28, Loss: 0.0000
Epoch 29, Loss: 0.0000
Epoch 30, Loss: 0.0000
Epoch 31, Loss: 0.0000
Epoch 32, Loss: 0.0000
Epoch 33, Loss: 0.0000
Epoch 34, Loss: 0.0000
Epoch 35, Loss: 0.0000
Epoch 36, Loss: 0.0000
Epoch 37, Loss: 0.0000
Epoch 38, Loss: 0.0000
Epoch 39, Loss: 0.0000
Epoch 40, Loss: 0.0000
Epoch 41, Loss: 0.0000
Epoch 42, Loss: 0.0000
Epoch 43, Loss: 0.0000
Epoch 44, Loss: 0.00

In [31]:
similarity_matrix_test = cosine_similarity(test_df)
threshold = 0.5  # Define threshold for creating edges

# Create edges based on similarity threshold
src_test, dst_test = [], []
for i in range(similarity_matrix_test.shape[0]):
    for j in range(i + 1, similarity_matrix_test.shape[1]):
        if similarity_matrix_test[i, j] > threshold:
            src_test.append(i)
            dst_test.append(j)


In [32]:
g_test = dgl.graph((src_test + dst_test, dst_test + src_test))  # Undirected graph

# Add node features (embeddings) to the graph
g_test.ndata['feat'] = torch.tensor(test_df.values, dtype=torch.float32)

num_nodes_test = test_df.shape[0]


In [33]:
model.eval()
with torch.no_grad():
    test_output = model(g_test, g_test.ndata['feat'])
    test_predictions = test_output.argmax(dim=1)  # Predicted classes

In [34]:
# Convert to NumPy for evaluation
predictions = test_predictions.numpy()
test_labels = test_labels.values


In [36]:
accuracy = accuracy_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.6818
F1 Score: 0.5528
